In [7]:
from sympy.tensor.array import tensorproduct, Array
from sympy import *
from random import randint
import numpy as np
import tensorly as tl
a11, a12, a21, a22,a23, a31, a32,a33, b11, b12, b21, b22,b23, b31, b32,b33= symbols('a11 a12 a21 a22 a23 a31 a32 a33 b11 b12 b21 b22 b23 b31 b32 b33')
c11, c12, c21, c22, c23, c31, c32, c33, pi_1, pi_2, pi_3= symbols('c11 c12 c21 c22 c23 c31 c32 c33 pi_1 pi_2 pi_3')
a13, b13, c13 = symbols('a13 b13 c13')
p_000, p_001, p_100, p_101, p_200, p_201, p_010, p_011, p_020, p_021, p_110, p_111, p_210, p_211, p_120, p_121, p_220, p_221 = symbols('p_000 p_001 p_100 p_101 p_200 p_201 p_010 p_011 p_020 p_021 p_110 p_111 p_210 p_211 p_120 p_121 p_220 p_221')
p_002, p_012, p_022, p_102, p_112, p_122, p_202, p_212, p_222 = symbols('p_002 p_012 p_022 p_102 p_112 p_122 p_202 p_212 p_222')
x_1, x_2, x_3 = symbols('x_1 x_2 x_3')

In [8]:
from tensorly.decomposition import parafac
from tensorly.decomposition import non_negative_parafac_hals
from tensorly.decomposition import non_negative_parafac

def check_rank(tensor, rank, non_neg=True, n=10, tol=0.001, p=False):
    if non_neg:
        for k in range(n):
            weights, factors = non_negative_parafac_hals(tensor, n_iter_max=1000000, rank=rank, init='random')
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                # print('doing', k)
                # print(full)
                print(tl.max(abs(diff)))
                print(factors)
            if tl.max(abs(diff)) < tol:
                return True
    else:
        for k in range(n):
            weights, factors = parafac(tensor, n_iter_max=1000000, rank=rank)
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                print(tl.max(abs(diff)))
                print(factors)
            if tl.max(abs(diff)) < tol:
                return True
    return False

In [9]:
def find_det(A):
    upper = A[0:2,0:2]
    lower = Matrix([A[2,0], A[2,1]]).transpose()
    B = Matrix([lower,lower * upper])
    return det(B)

In [10]:
high = 80
def low_tensor_spec():
    a = np.random.randint(1, high/2, size=2) * 0.1
    b = np.random.randint(1, high, size=3)  * 1.0
    c = np.random.randint(1, high, size=3) * 1.0
    #c[1] = -c[1]
    #c[0] = -c[0]
    a[1] = -3.1
    a[0] = -4.2
    #print(a,b,c)
    tens = tl.tensor(np.kron(np.kron(a, b), c).reshape(2, 3, 3)) * 1.0
    return (tens,a,b,c)

def low_tensor_spec1():
    a = np.random.randint(1, high/2, size=2) * 0.1
    b = np.random.randint(1, high, size=3)  * 1.0
    c = np.random.randint(1, high, size=3) *1.0
    a[0] = 0
    #a[1] = -a[1]
    #a[1] = 32
    #a[1] = 0
    #a[0] = 0
    #c[1] = -c[1]
    #c[0] = 0
    #print(a,b,c)
    tens = tl.tensor(np.kron(np.kron(a, b), c).reshape(2, 3, 3)) * 1.0
    return (tens,a,b,c)

def low_tensor():
    a = np.random.randint(1, high, size=2) * 1.0
    b = np.random.randint(1, high, size=3) * 1.0
    c = np.random.randint(1, high, size=3) * 1.0
    #a= a/sum(a)
    #b = b/sum(b)
    #c= c/sum(c)
    tens = tl.tensor(np.kron(np.kron(a, b), c).reshape(2, 3, 3)) * 1.0
    #print(a,b,c)
    return (tens,a,b,c)


In [11]:
t1 = low_tensor()
t2 = low_tensor()
t3 = low_tensor()
t4 = low_tensor()
t5 = low_tensor()
a1 = Matrix([Array(t1[2]), Array(t2[2]), Array(t3[2])]).transpose()
a2 = Matrix([Array(t1[3]), Array(t2[3]), Array(t3[3])]).transpose()
a3 = t1[1]
b3 = t2[1]
c3 = t3[1]
tens = t1[0]*0.1 +t2[0]*0.5 + t3[0]
#tens = np.random.randint(1,100000,size = (2,3,3))
#tens
t01 = Matrix(tens[0])
t02 = Matrix(tens[1])

tens[0] = t01
tens[1] = t02
t11 = Matrix([tens[0][0],tens[0][2],tens[0][1]])
t12 = Matrix([tens[1][0],tens[1][2],tens[1][1]])

t21 = Matrix([tens[0][2],tens[0][1],tens[0][0]])
t22 = Matrix([tens[1][2],tens[1][1],tens[1][0]])

cs = (a3[0] * b3[1] - a3[1] *b3[0])*(a3[0] * c3[1] - a3[1] * c3[0])*(b3[0] * c3[1] - c3[0] * b3[1])
A1a = t01 * t02.adjugate()
A2a = -t11 * t12.adjugate()
A3a = -t21 * t22.adjugate()
inver = abs(det(t02)) > 10 and abs(det(t01)) > 10
if inver:
    A1 = t01*t02.inv() #/(abs(det(t02))/det(t02))
    A2 = t11*t12.inv() #/(abs(det(t12))/det(t12))
    A3 = t21 * t22.inv() #/(abs(det(t22))/det(t22))
    #print(find_det(A1),find_det(A2),find_det(A3))
    
print(find_det(A1a),find_det(A2a),find_det(A3a))
print(det(t01), det(t02), t02.minor(2,2), t02.minor(1,1), t02.minor(0,0))
#print(t11.eigenvects())
#print(t12.eigenvects())
prod1 = a1[2,0] * a1[2,1] * a1[2,2]
prod2 = a1[1,0] * a1[1,1] * a1[1,2]
prod3 = a1[0,0] * a1[0,1] * a1[0,2]
r1 = cs * det(a1) * det(a2)**2 *prod1 #/ -det(t02)**2
r2 = cs * -det(a1) * det(a2)**2 *prod2#/ -det(t02)**2
r3 = cs * -det(a1) * det(a2)**2 *prod3# / -det(t02)**2
t01 = Matrix(tens[0]).transpose()
t02 = Matrix(tens[1]).transpose()
tens[0] = t01
tens[1] = t02
t11 = Matrix([tens[0][0],tens[0][2],tens[0][1]])
t12 = Matrix([tens[1][0],tens[1][2],tens[1][1]])
t21 = Matrix([tens[0][2],tens[0][1],tens[0][0]])
t22 = Matrix([tens[1][2],tens[1][1],tens[1][0]])
if inver:
    A11 = t01*t02.inv() #/(abs(det(t02))/det(t02))
    A21 = t11*t12.inv() #/(abs(det(t12))/det(t12))
    A31 = t21 * t22.inv() #/(abs(det(t22))/det(t22))
    #print(find_det(A11),find_det(A21),find_det(A31))
    
A11a = t01 * t02.adjugate() #/(abs(det(t02))/det(t02))
A21a = -t11 * t12.adjugate() #/(abs(det(t12))/det(t12))
A31a = -t21 * t22.adjugate() #/(abs(det(t22))/det(t22))
print(find_det(A11a),find_det(A21a),find_det(A31a))
#print(r1,r2,r3)
tens

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [17]:
Matrix(tens[0][0])

Matrix([
[128150.0],
[321543.0],
[350434.0]])

In [124]:
A1a.eigenvects()

[(474488799523.053,
  1,
  [Matrix([
   [0.293380711889363],
   [0.497471641899339],
   [0.816363720039828]])]),
 (-0.127400853230118,
  1,
  [Matrix([
   [-0.563311175619855],
   [ -0.61826836348582],
   [-0.645746957419894]])]),
 (112708483171.012,
  1,
  [Matrix([
   [-0.119446056612228],
   [-0.371609953905703],
   [-0.955568452901191]])])]

In [125]:
M = A11a.eigenvects()
M

[(0.0327829423219696,
  1,
  [Matrix([
   [ 0.05362703388568],
   [0.706089279495332],
   [0.706089279495438]])]),
 (112708483171.272,
  1,
  [Matrix([
   [-0.976477766031913],
   [-0.390591106413513],
   [-0.927653877731321]])]),
 (474488799522.625,
  1,
  [Matrix([
   [-0.586209097577232],
   [ -0.45724309611023],
   [-0.914486192220423]])])]

In [111]:
M = A1a.eigenvects()
M1 = abs(Matrix([[Matrix(M[0][2]),Matrix(M[1][2]),Matrix(M[2][2])]])).transpose()
M1
K = A11a.eigenvects()
K1 = abs(Matrix([[Matrix(K[0][2]),Matrix(K[1][2]),Matrix(K[2][2])]]))
K1.inv() * Matrix(tens[1]) * M1.inv()

Matrix([
[ 0.00268015265464783, 1414683.78650759, -1409277.68553178],
[ 0.00394018739461899, 1769837.53198448, -2073062.82271748],
[-0.00681251287460327, -3060300.4368331,  3584620.02712829]])

In [112]:
K1.inv() * Matrix(tens[0]) * M1.inv()

Matrix([
[ 0.00012505054473877,  3104443.79869714, -53344.3249796629],
[0.000162839889526367,  4034432.04488708, -69324.5128810406],
[    1034.32029461861, -6976105.95673348,   119871.92791152]])

In [8]:
K = t01 * t12.inv()
K.eigenvects()

NameError: name 't12' is not defined

In [9]:
def find_det1(A):
    #lower = Matrix([A[2,0], A[2,1]]).transpose()
    #B = Matrix([lower,lower * upper])
    return (A[1,0],A[2,0])

In [210]:
t1 = low_tensor()
t2 = low_tensor_spec()
t3 = low_tensor_spec1()
t4 = low_tensor()
t5 = low_tensor()
tens = t1[0] -t2[0]*0.1 -t3[0]*0.1
t01 = Matrix(tens[0])
t02 = Matrix(tens[1])
print(det(t01), det(t02),t01.minor(2,2),t01.minor(1,1),t01.minor(0,0),t02.minor(2,2),t02.minor(1,1),t02.minor(0,0))
A = t01 * t02.adjugate()
print(A.eigenvects())
tens

-6.00000000000000 0 202394155.040001 92765608.3200073 666491696.639999 -80409288.9599991 -27911869.4400024 -338775557.119995
[(1.28472780776442, 1, [Matrix([
[-0.388885451763465],
[-0.160318315979255],
[-0.907229928390979]])]), (0.000657946440063254, 1, [Matrix([
[-0.0578938377835887],
[ 0.0876258271578048],
[  -1.09292465201625]])]), (57096096977.9060, 1, [Matrix([
[0.590146956813343],
[0.490275933345675],
[0.762651451899074]])])]


array([[[ 250331.6 ,  462350.6 ,  160572.  ],
        [ 102513.76,  190146.56,   65606.4 ],
        [ 584480.96, 1077507.76,  375278.4 ]],

       [[ 132117.44,  238261.6 ,   85124.16],
        [  54337.92,   97384.8 ,   35034.88],
        [ 309312.64,  563045.6 ,  199080.96]]])

[(8.07692307692299,
  1,
  [Matrix([
   [-0.925305138660671],
   [-0.364054480784529],
   [-0.106182556895493]])]),
 (0.333333333333430,
  1,
  [Matrix([
   [-0.765729428086548],
   [-0.592164091053593],
   [-0.714680799547438]])]),
 (93.9999999999999,
  1,
  [Matrix([
   [-0.886872630879788],
   [-0.147812105146632],
   [ -1.18249684117305]])])]